In [2]:
import warnings
warnings.filterwarnings('ignore')

import glob
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import sklearn.preprocessing as skp

from gan_models import GeneratorResNet
from scipy.interpolate import splrep, splev
from biosppy.signals import tools as tools
from tqdm import tqdm

import heartpy as hp
from sklearn.metrics import mean_absolute_error
import neurokit2 as nk

- Define Device

In [3]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print("Using Pytorch Versions:", torch.__version__, ' Device:', DEVICE)

Using Pytorch Versions: 2.1.1+cu118  Device: cuda


- 필요 함수

In [4]:
def interp_spline(ecg, step=1, k=3):

    x_new = np.arange(0, ecg.shape[0], ecg.shape[0]/step)
    interp_spline_method = splrep(np.arange(0, ecg.shape[0], 1), ecg, k=k)
    return splev(x_new, interp_spline_method)

In [5]:
def filter_ppg(signal, sampling_rate):
    
    signal = np.array(signal)
    sampling_rate = float(sampling_rate)
    filtered, _, _ = tools.filter_signal(signal=signal,
                                  ftype='butter',
                                  band='bandpass',
                                  order=4, #3
                                  frequency=[1, 8], #[0.5, 8]
                                  sampling_rate=sampling_rate)

    return filtered

In [6]:
def calc_hr(peaks, fs=250):
    rr_inter = []
    tmp = 0
    for x in peaks:
        if tmp == 0:
            tmp = x
            continue
        rr_inter.append(60 / (abs(x - tmp) / fs))
        tmp = x
    return np.mean(rr_inter)

def calc_ppg_hr(ppg_sig, fs, peak_method='elgendi'):
    """
    method : "elgendi", "bishop"
    """
    ppg_peak = nk.ppg_findpeaks(ppg_sig, sampling_rate=fs, method=peak_method)['PPG_Peaks']
    hr = calc_hr(ppg_peak, fs)
    
    return hr

def calc_ecg_hr(ecg_sig, fs, peak_method='nabian2018'):
    """
    method : "nabian2018", "neurokit"
    """
    ecg_peak = nk.ecg_peaks(ecg_sig, sampling_rate=fs, method="nabian2018")[1]['ECG_R_Peaks']
    hr = calc_hr(ecg_peak, fs)
    
    return hr

In [7]:
def MAE(y, y_pred):
    return mean_absolute_error(y, y_pred)

In [8]:
def load_pickle(path):
    with open(path,'rb') as file: # Binary read
        _data = pickle._Unpickler(file)
        _data.encoding = 'latin1'
        data = _data.load()
    return data

- load P2E model

In [9]:
weights_path = './gan_weights/PPG2ECG_CycleGAN_1Epochs.pth'
input_shape = (None, 1, int(128 * 64))
n_residual_blocks = 6

G_AB = GeneratorResNet(input_shape, n_residual_blocks)
weights = torch.load(weights_path)
G_AB.load_state_dict(weights['G_AB'])
G_AB.to(DEVICE)
G_AB.eval()

GeneratorResNet(
  (model): Sequential(
    (0): ReflectionPad1d((1, 1))
    (1): Conv1d(1, 64, kernel_size=(7,), stride=(1,), padding=(1,))
    (2): InstanceNorm1d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Conv1d(64, 128, kernel_size=(3,), stride=(2,), padding=(2,))
    (5): InstanceNorm1d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): LeakyReLU(negative_slope=0.2, inplace=True)
    (7): Conv1d(128, 256, kernel_size=(3,), stride=(2,), padding=(2,))
    (8): InstanceNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): LeakyReLU(negative_slope=0.2, inplace=True)
    (10): ResidualBlock(
      (block): Sequential(
        (0): ReflectionPad1d((1, 1))
        (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
        (2): InstanceNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): LeakyReLU(neg

- get datapaths

In [10]:
sig_paths = glob.glob('../00_Data/01_PPG2ECG/01_Original/04_WESAD/*/*.pkl')
gt_paths = [i for i in range(len(sig_paths))]

- HR 추정

In [12]:
ppg_origin_sig_fs = 64
ecg_origin_sig_fs = 700
target_sig_fs = 128
window_sec = 8
batch_size = 100

ppg_window_size = window_sec * ppg_origin_sig_fs
ecg_window_size = window_sec * ecg_origin_sig_fs

ppg_hr_mae_list = []
ecg_hr_mae_list = []
for sig_path, gt_path in tqdm(zip(sig_paths, gt_paths), total=len(sig_paths)):
    origin_sig = load_pickle(sig_path)
#     origin_gt = pd.read_csv(gt_path)
    
    ppg_sig = origin_sig['signal']['wrist']['BVP'].flatten()
    origin_ecg_sig = origin_sig['signal']['chest']['ECG'].flatten()
    
    # PPG signal process
    ppg_sig_processed = []
    sig_iters = len(ppg_sig) // ppg_window_size
    for i in range(sig_iters):
        ppg_seg = ppg_sig[i*ppg_window_size : (i+1)*ppg_window_size]
        ppg_seg = interp_spline(ppg_seg, step=target_sig_fs*window_sec, k=5)
        ppg_seg = (ppg_seg-ppg_seg.mean()) / (ppg_seg.std() + 1e-17)
        ppg_seg = skp.minmax_scale(ppg_seg, (-1, 1), axis=0)
        ppg_sig_processed.append(np.expand_dims(ppg_seg, 0))
    ppg_sig_processed = np.array(ppg_sig_processed)
    
    # Origin ECG signal process (do not interpolation)
    origin_ecg_sig_processed = []
    sig_iters = len(origin_ecg_sig) // ecg_window_size
    for i in range(sig_iters):
        origin_ecg_seg = origin_ecg_sig[i*ecg_window_size : (i+1)*ecg_window_size]
        origin_ecg_seg = (origin_ecg_seg-origin_ecg_seg.mean()) / (origin_ecg_seg.std() + 1e-17)
        origin_ecg_seg = skp.minmax_scale(origin_ecg_seg, (-1, 1), axis=0)
        origin_ecg_sig_processed.append(np.expand_dims(origin_ecg_seg, 0))
    origin_ecg_sig_processed = np.array(origin_ecg_sig_processed)
    
    # make syn ecg (batch size)
    input_ppg_sig = torch.from_numpy(ppg_sig_processed).type(torch.FloatTensor)
    syn_ecg_sig = []
    
    infer_iters = len(input_ppg_sig) // batch_size
    for i in range(infer_iters):
        batch_input_ppg = input_ppg_sig[i*batch_size : (i+1)*batch_size].to(DEVICE)
        batch_syn_ecg = G_AB(batch_input_ppg).data.cpu().numpy()
        syn_ecg_sig.extend(batch_syn_ecg)
        
    syn_ecg_sig = np.array(syn_ecg_sig)
    origin_ecg_sig_processed = origin_ecg_sig_processed[:syn_ecg_sig.shape[0]]
    ppg_sig_processed = ppg_sig_processed[:syn_ecg_sig.shape[0]]      
    
    # HR GT process (origin ecg -> HR)
    HR_GT_seg = []
    for i in range(len(origin_ecg_sig_processed)):
        try:
            gt_hr = calc_ecg_hr(origin_ecg_sig_processed[i][0], fs=ecg_origin_sig_fs)
            HR_GT_seg.append(gt_hr)
        except:
            HR_GT_seg.append(np.NaN)
    HR_GT_seg = np.array(HR_GT_seg)
        
    # inference PPG HR
    ppg_infer_hr = []
    for i in range(len(ppg_sig_processed)):
        try:
            ppg_hr = calc_ppg_hr(ppg_sig_processed[i][0], fs=target_sig_fs)
            ppg_infer_hr.append(ppg_hr)
        except:
            ppg_infer_hr.append(np.NaN)
    ppg_infer_hr = np.array(ppg_infer_hr)
    
    # inference SYNECG HR
    ecg_infer_hr = []
    for i in range(len(ppg_sig_processed)):
        try:
            ecg_hr = calc_ppg_hr(syn_ecg_sig[i][0], fs=target_sig_fs)
            ecg_infer_hr.append(ecg_hr)
        except:
            ecg_infer_hr.append(np.NaN)
    ecg_infer_hr = np.array(ecg_infer_hr)
        
    # calculate MAE (NaN 제거)
    if np.isnan(HR_GT_seg.sum()) or np.isnan(ppg_infer_hr.sum()) or np.isnan(ecg_infer_hr.sum()):
        except_index = np.argwhere(np.isnan(HR_GT_seg)).flatten()
        HR_GT_seg = np.delete(HR_GT_seg, except_index)
        ppg_infer_hr = np.delete(ppg_infer_hr, except_index)
        ecg_infer_hr = np.delete(ecg_infer_hr, except_index)
        
        except_index = np.argwhere(np.isnan(ppg_infer_hr)).flatten()
        HR_GT_seg = np.delete(HR_GT_seg, except_index)
        ppg_infer_hr = np.delete(ppg_infer_hr, except_index)
        ecg_infer_hr = np.delete(ecg_infer_hr, except_index)
        
        except_index = np.argwhere(np.isnan(ecg_infer_hr)).flatten()
        HR_GT_seg = np.delete(HR_GT_seg, except_index)
        ppg_infer_hr = np.delete(ppg_infer_hr, except_index)
        ecg_infer_hr = np.delete(ecg_infer_hr, except_index)
            
    ppg_mae = MAE(HR_GT_seg, ppg_infer_hr)
    ecg_mae = MAE(HR_GT_seg, ecg_infer_hr)

    ppg_hr_mae_list.append(ppg_mae)
    ecg_hr_mae_list.append(ecg_mae)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [04:12<00:00, 16.84s/it]


- PPG HR 오차

In [13]:
np.array(ppg_hr_mae_list).mean()

11.86353078954456

- SYNECG HR 오차

In [14]:
np.array(ecg_hr_mae_list).mean()

9.428658128403312